<a href="https://colab.research.google.com/github/juanangelgomez22/7506R-1C2023-GRUPO08-GOMEZ/blob/main/TA047R_TP1_GRUPO02_ENTREGA_EJ1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Organización de Datos**

Trabajo Práctico N°1: TLC Trip Record Data

Grupo 02 - 2C2024

Cátedra Ing. Rodriguez, Juan Manuel

##Ruta del archivo a procesar

importo librerias a usar y vinculo al drive

In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
from matplotlib import pyplot as plt
from scipy.stats import chi2_contingency
from datetime import time
import pyarrow.parquet as pq
from datetime import date
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
##df = pq.read_table('dataset.parq').to_pandas()

from google.colab import drive
drive.mount('/content/drive')

#taxis_train=pd.read_csv('/content/drive/MyDrive/Ta047-tp_2do_cuat_2024_grupo_2/Data/EJ1/taxi_zone_lookup.csv')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


##descripcion del tipo de datos almacenados en los datasets de fechas a analizar
* Los registros de viajes en taxi amarillo y verde incluyen campos que registran las fechas y horas de recogida y entrega, los lugares de recogida y entrega, las distancias de los viajes, las tarifas detalladas, los tipos de tarifas, los tipos de pago y los recuentos de pasajeros informados por el conductor. Los datos utilizados en los conjuntos de datos adjuntos fueron recopilados y proporcionados a la Comisión de Taxis y Limusinas de la Ciudad de Nueva York (TLC) por proveedores de tecnología autorizados en virtud de los Programas de Mejora de Pasajeros de Taxis y Limusinas (TPEP/LPEP). Los datos de los viajes no fueron creados por la TLC, y la TLC no hace declaraciones sobre la precisión de estos datos.

* Los registros de viajes de vehículos de alquiler (“FHV”) incluyen campos que capturan el número de licencia de la base de despacho y la fecha, hora y ubicación de la zona de taxis de recogida (archivo de forma a continuación). Estos registros se generan a partir de los envíos de registros de viajes de FHV realizados por las bases.

    Nota: La TLC publica los datos de los registros de viajes de las bases tal como los envían las bases y no podemos garantizar ni confirmar su precisión o integridad. Por lo tanto, es posible que esto no represente la cantidad total de viajes enviados por todas las bases con licencia de la TLC. La TLC realiza revisiones de rutina de los registros y toma medidas de cumplimiento cuando es necesario para garantizar, en la medida de lo posible, información completa y precisa.
    
* Los archivos HVFHV ahora incluirán 17 columnas más (consulte el Diccionario de viajes de FHV de alto volumen para obtener más detalles). También se agregarán columnas adicionales a los archivos antiguos. La fecha más temprana para incluir columnas adicionales: febrero de 2019.

* Los datos de viajes en amarillo incluirán ahora una columna adicional ('airport_fee', consulte el Diccionario de viajes en amarillo para obtener más detalles). La columna adicional también se agregará a los archivos antiguos. La fecha más temprana en la que se incluirá la columna adicional: enero de 2011.

##periodos elegidos/Asignados
Abril, Mayo y Junio

***1) Registro de viajes en taxi amarillo (Yellow Taxi Trip Records)***

  a) VendorID: Un código que indica el proveedor de TPEP que proporcionó el registro.
    1= Creative Mobile Technologies, LLC; 2= VeriFone Inc.

  b) tpep_pickup_datetime: La fecha y hora en que se activó el medidor.

  c) tpep_dropoff_datetime: La fecha y hora en que se desconectó el medidor.

  d) Passenger_count: El número de pasajeros en el vehículo. Este es un valor ingresado por el conductor.

  e) Trip_distance: La distancia recorrida del viaje en millas reportada por el taxímetro.

  f) PULocationID: Zona de Taxi TLC en la que estaba activado el taxímetro.

  g) DOLocationID: Zona de Taxi TLC en la que se desactivó el taxímetro.

  h) RateCodeID: El código de tarifa final vigente al final del viaje (pasar a entero).
    1= Standard rate
    2=JFK
    3=Newark
    4=Nassau or Westchester
    5=Negotiated fare
    6=Group ride

  i) Store_and_fwd_flag: Este indicador indica si el registro de viaje se mantuvo en la memoria del vehículo antes de enviarlo al proveedor, también conocido como "almacenar y reenviar". porque el vehículo no tenía conexión con el servidor.
    Y = viaje de ida y vuelta
    N = no es un viaje de ida y vuelta

  j) Payment_type: Un código numérico que indica cómo el pasajero pagó el viaje.
    1= Credit card
    2= Cash
    3= No charge
    4= Dispute
    5= Unknown
    6= Voided trip

  k) Fare_amount: La tarifa por tiempo y distancia calculada por el taxímetro.

  l) Extra: Extras y recargos varios. Actualmente, esto sólo incluye los cargos de $0,50  y $1,00 por hora pico y por noche.

  m) MTA_tax: Impuesto MTA de $0.50 que se activa automáticamente según la tarifa medida en uso.

  ñ) Improvement_surcharge: Recargo por mejora de $0.30 en viajes evaluados al bajar la bandera. El recargo por mejora comenzó a cobrarse en 2015 (pasar a entero).

  o) Tip_amount: Monto de la propina: este campo se completa automáticamente para las propinas de tarjetas de crédito. Las propinas en efectivo no están incluidas.

  p) Tolls_amount: Importe total de todos los peajes pagados en el viaje.
  
  q) Total_amount: El importe total cobrado a los pasajeros. No incluye propinas en efectivo.
  
  r) Congestion_Surcharge: Monto total cobrado en el viaje por el recargo por congestión del Estado de Nueva York.
  
  s) Airport_fee: $1.25 para recogida únicamente en los aeropuertos LaGuardia y John F. Kennedy


  ## * Taxis Amarillos de abril

In [2]:
taxis_amarillos_abril=pd.read_parquet('/content/drive/MyDrive/Ta047-tp_2do_cuat_2024_grupo_2/Data/EJ1/datos/abril/yellow_tripdata_2023-04.parquet')
taxis_amarillos_abril

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee
0,1,2023-04-01 00:14:49,2023-04-01 00:45:01,2.0,4.90,1.0,N,48,223,1,28.90,3.5,0.5,6.00,0.00,1.0,39.90,2.5,0.00
1,2,2023-04-01 00:00:24,2023-04-01 00:56:19,1.0,21.89,2.0,N,132,43,2,70.00,0.0,0.5,0.00,6.55,1.0,81.80,2.5,1.25
2,1,2023-04-01 00:03:50,2023-04-01 00:14:42,2.0,1.30,1.0,N,148,113,1,11.40,3.5,0.5,2.00,0.00,1.0,18.40,2.5,0.00
3,1,2023-04-01 00:53:18,2023-04-01 01:01:28,1.0,1.50,1.0,N,249,79,1,10.00,3.5,0.5,1.00,0.00,1.0,16.00,2.5,0.00
4,2,2023-04-01 00:07:00,2023-04-01 00:17:16,2.0,1.49,1.0,N,158,246,1,11.40,1.0,0.5,1.00,0.00,1.0,17.40,2.5,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3288245,2,2023-04-30 23:50:19,2023-05-01 00:06:11,NaN,2.02,NaN,None,246,79,0,11.41,0.0,0.5,3.08,0.00,1.0,18.49,NaN,NaN
3288246,2,2023-04-30 23:28:32,2023-04-30 23:38:32,NaN,2.36,NaN,None,114,68,0,13.19,0.0,0.5,0.00,0.00,1.0,17.19,NaN,NaN
3288247,2,2023-04-30 23:25:26,2023-04-30 23:44:16,NaN,4.64,NaN,None,137,74,0,17.90,0.0,0.5,0.00,0.00,1.0,21.90,NaN,NaN
3288248,1,2023-04-30 23:11:59,2023-04-30 23:26:19,NaN,0.00,NaN,None,162,151,0,16.00,1.0,0.5,4.20,0.00,1.0,27.20,NaN,NaN


## * Amarillos de mayo

In [3]:
taxis_amarillos_mayo=pd.read_parquet('/content/drive/MyDrive/Ta047-tp_2do_cuat_2024_grupo_2/Data/EJ1/datos/mayo/yellow_tripdata_2023-05.parquet')
taxis_amarillos_mayo

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee
0,1,2023-05-01 00:33:13,2023-05-01 00:53:01,0.0,7.80,1.0,N,138,43,1,33.80,7.75,0.5,8.60,0.00,1.0,51.65,0.0,1.75
1,1,2023-05-01 00:42:49,2023-05-01 01:11:18,2.0,8.10,1.0,N,138,262,1,35.90,10.25,0.5,9.50,0.00,1.0,57.15,2.5,1.75
2,1,2023-05-01 00:56:34,2023-05-01 01:13:39,2.0,9.10,1.0,N,138,141,1,35.20,10.25,0.5,10.70,6.55,1.0,64.20,2.5,1.75
3,2,2023-05-01 00:00:52,2023-05-01 00:20:12,1.0,8.21,1.0,N,138,140,1,33.10,6.00,0.5,2.24,0.00,1.0,47.09,2.5,1.75
4,1,2023-05-01 00:05:50,2023-05-01 00:19:41,0.0,7.90,1.0,N,138,263,1,31.00,10.25,0.5,9.85,6.55,1.0,59.15,2.5,1.75
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3513644,2,2023-05-31 23:59:36,2023-06-01 00:13:11,NaN,2.18,NaN,None,186,229,0,13.56,0.00,0.5,3.51,0.00,1.0,21.07,NaN,NaN
3513645,2,2023-05-31 23:52:44,2023-05-31 23:56:07,NaN,0.67,NaN,None,224,137,0,11.94,0.00,0.5,2.23,0.00,1.0,18.17,NaN,NaN
3513646,2,2023-05-31 23:26:09,2023-05-31 23:26:23,NaN,0.00,NaN,None,161,161,0,10.63,0.00,0.5,2.93,0.00,1.0,17.56,NaN,NaN
3513647,2,2023-05-31 23:03:44,2023-05-31 23:42:29,NaN,13.95,NaN,None,162,259,0,45.52,0.00,0.5,7.00,0.00,1.0,56.52,NaN,NaN


  ## * Taxis Amarillos de junio

In [4]:
taxis_amarillos_junio=pd.read_parquet('/content/drive/MyDrive/Ta047-tp_2do_cuat_2024_grupo_2/Data/EJ1/datos/junio/yellow_tripdata_2023-06.parquet')
taxis_amarillos_junio

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee
0,1,2023-06-01 00:08:48,2023-06-01 00:29:41,1.0,3.40,1.0,N,140,238,1,21.90,3.50,0.5,6.70,0.00,1.0,33.60,2.5,0.00
1,1,2023-06-01 00:15:04,2023-06-01 00:25:18,0.0,3.40,1.0,N,50,151,1,15.60,3.50,0.5,3.00,0.00,1.0,23.60,2.5,0.00
2,1,2023-06-01 00:48:24,2023-06-01 01:07:07,1.0,10.20,1.0,N,138,97,1,40.80,7.75,0.5,10.00,0.00,1.0,60.05,0.0,1.75
3,2,2023-06-01 00:54:03,2023-06-01 01:17:29,3.0,9.83,1.0,N,100,244,1,39.40,1.00,0.5,8.88,0.00,1.0,53.28,2.5,0.00
4,2,2023-06-01 00:18:44,2023-06-01 00:27:18,1.0,1.17,1.0,N,137,234,1,9.30,1.00,0.5,0.72,0.00,1.0,15.02,2.5,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3307229,2,2023-06-30 23:30:21,2023-07-01 00:00:46,NaN,9.20,NaN,None,42,148,0,38.53,0.00,0.5,8.51,0.00,1.0,51.04,NaN,NaN
3307230,1,2023-06-30 23:34:22,2023-07-01 00:32:59,NaN,20.20,NaN,None,132,74,0,70.00,1.75,0.5,11.97,6.55,1.0,91.77,NaN,NaN
3307231,2,2023-06-30 23:45:00,2023-07-01 00:10:00,NaN,6.16,NaN,None,256,140,0,28.03,0.00,0.5,6.41,0.00,1.0,38.44,NaN,NaN
3307232,2,2023-06-30 23:13:38,2023-06-30 23:51:19,NaN,11.65,NaN,None,91,246,0,45.74,0.00,0.5,0.00,6.55,1.0,56.29,NaN,NaN


##reviso tipos de datos de cada columna

In [5]:
taxis_amarillos_abril.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3288250 entries, 0 to 3288249
Data columns (total 19 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   VendorID               int32         
 1   tpep_pickup_datetime   datetime64[us]
 2   tpep_dropoff_datetime  datetime64[us]
 3   passenger_count        float64       
 4   trip_distance          float64       
 5   RatecodeID             float64       
 6   store_and_fwd_flag     object        
 7   PULocationID           int32         
 8   DOLocationID           int32         
 9   payment_type           int64         
 10  fare_amount            float64       
 11  extra                  float64       
 12  mta_tax                float64       
 13  tip_amount             float64       
 14  tolls_amount           float64       
 15  improvement_surcharge  float64       
 16  total_amount           float64       
 17  congestion_surcharge   float64       
 18  Airport_fee           

## veo cantidad de nulos en abril
veo cuantos nulos hay por cada columna de la tabla de abril.
si son pocos solo los borro

In [6]:
print("cantidad de no nulos en abril")
print(taxis_amarillos_abril.count()-taxis_amarillos_abril.isna().sum())

cantidad de no nulos en abril
VendorID                 3288250
tpep_pickup_datetime     3288250
tpep_dropoff_datetime    3288250
passenger_count          3106870
trip_distance            3288250
RatecodeID               3106870
store_and_fwd_flag       3106870
PULocationID             3288250
DOLocationID             3288250
payment_type             3288250
fare_amount              3288250
extra                    3288250
mta_tax                  3288250
tip_amount               3288250
tolls_amount             3288250
improvement_surcharge    3288250
total_amount             3288250
congestion_surcharge     3106870
Airport_fee              3106870
dtype: int64


## veo cantidad de nulos en abril
veo cuantos nulos hay por cada columna de la tabla de mayo.
si son pocos solo los borro

In [7]:
print("cantidad de no nulos en mayo")
print(taxis_amarillos_mayo.count()-taxis_amarillos_mayo.isna().sum())

cantidad de no nulos en mayo
VendorID                 3513649
tpep_pickup_datetime     3513649
tpep_dropoff_datetime    3513649
passenger_count          3310057
trip_distance            3513649
RatecodeID               3310057
store_and_fwd_flag       3310057
PULocationID             3513649
DOLocationID             3513649
payment_type             3513649
fare_amount              3513649
extra                    3513649
mta_tax                  3513649
tip_amount               3513649
tolls_amount             3513649
improvement_surcharge    3513649
total_amount             3513649
congestion_surcharge     3310057
Airport_fee              3310057
dtype: int64


## veo cantidad de nulos en junio
veo cuantos nulos hay por cada columna de la tabla de junio.
si son pocos solo los borro

In [8]:
print("cantidad de no nulos en junio")
print(taxis_amarillos_junio.count()-taxis_amarillos_junio.isna().sum())

cantidad de no nulos en junio
VendorID                 3307234
tpep_pickup_datetime     3307234
tpep_dropoff_datetime    3307234
passenger_count          3107460
trip_distance            3307234
RatecodeID               3107460
store_and_fwd_flag       3107460
PULocationID             3307234
DOLocationID             3307234
payment_type             3307234
fare_amount              3307234
extra                    3307234
mta_tax                  3307234
tip_amount               3307234
tolls_amount             3307234
improvement_surcharge    3307234
total_amount             3307234
congestion_surcharge     3107460
Airport_fee              3107460
dtype: int64


## Saco los nans##

In [9]:
taxis_amarillos_abril_limpio=taxis_amarillos_abril.dropna(axis = 0)
#convierto los float a enteros de 8
taxis_amarillos_abril_limpio['passenger_count']=taxis_amarillos_abril_limpio['passenger_count'].astype('int8')
taxis_amarillos_abril_limpio['RatecodeID']=taxis_amarillos_abril_limpio['RatecodeID'].astype('int8')
taxis_amarillos_abril_limpio['improvement_surcharge']=taxis_amarillos_abril_limpio['improvement_surcharge'].astype('int8')
#taxis_amarillos_abril_limpio['store_and_fwd_flag']=taxis_amarillos_abril_limpio['store_and_fwd_flag'].astype(bool)
taxis_amarillos_abril_limpio.loc[taxis_amarillos_abril_limpio.loc[:,"store_and_fwd_flag"]=='N',"store_and_fwd_flag"]=0
taxis_amarillos_abril_limpio.loc[taxis_amarillos_abril_limpio.loc[:,"store_and_fwd_flag"]=='Y',"store_and_fwd_flag"]=1
#taxis_amarillos_abril_limpio

<ipython-input-9-831dcacc32f1>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  taxis_amarillos_abril_limpio['passenger_count']=taxis_amarillos_abril_limpio['passenger_count'].astype('int8')
<ipython-input-9-831dcacc32f1>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  taxis_amarillos_abril_limpio['RatecodeID']=taxis_amarillos_abril_limpio['RatecodeID'].astype('int8')
<ipython-input-9-831dcacc32f1>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc

In [10]:
taxis_amarillos_mayo_limpio=taxis_amarillos_mayo.dropna(axis = 0)
taxis_amarillos_mayo_limpio['passenger_count']=taxis_amarillos_mayo_limpio['passenger_count'].astype('int8')
taxis_amarillos_mayo_limpio['RatecodeID']=taxis_amarillos_mayo_limpio['RatecodeID'].astype('int8')
taxis_amarillos_mayo_limpio['improvement_surcharge']=taxis_amarillos_mayo_limpio['improvement_surcharge'].astype('int8')
#taxis_amarillos_mayo_limpio['store_and_fwd_flag']=taxis_amarillos_mayo_limpio['store_and_fwd_flag'].astype(bool)
taxis_amarillos_mayo_limpio.loc[taxis_amarillos_mayo_limpio.loc[:,"store_and_fwd_flag"]=='N',"store_and_fwd_flag"]=0
taxis_amarillos_mayo_limpio.loc[taxis_amarillos_mayo_limpio.loc[:,"store_and_fwd_flag"]=='Y',"store_and_fwd_flag"]=1
#taxis_amarillos_mayo_limpio

<ipython-input-10-fe41bebc9633>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  taxis_amarillos_mayo_limpio['passenger_count']=taxis_amarillos_mayo_limpio['passenger_count'].astype('int8')
<ipython-input-10-fe41bebc9633>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  taxis_amarillos_mayo_limpio['RatecodeID']=taxis_amarillos_mayo_limpio['RatecodeID'].astype('int8')
<ipython-input-10-fe41bebc9633>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[

In [11]:
taxis_amarillos_junio_limpio=taxis_amarillos_junio.dropna(axis = 0)
taxis_amarillos_junio_limpio['passenger_count']=taxis_amarillos_junio_limpio['passenger_count'].astype('int8')
taxis_amarillos_junio_limpio['RatecodeID']=taxis_amarillos_junio_limpio['RatecodeID'].astype('int8')
taxis_amarillos_junio_limpio['improvement_surcharge']=taxis_amarillos_junio_limpio['improvement_surcharge'].astype('int8')
#taxis_amarillos_junio_limpio['store_and_fwd_flag']=taxis_amarillos_junio_limpio['store_and_fwd_flag'].astype(bool)
taxis_amarillos_junio_limpio.loc[taxis_amarillos_junio_limpio.loc[:,"store_and_fwd_flag"]=='N',"store_and_fwd_flag"]=0
taxis_amarillos_junio_limpio.loc[taxis_amarillos_junio_limpio.loc[:,"store_and_fwd_flag"]=='Y',"store_and_fwd_flag"]=1
taxis_amarillos_junio_limpio

<ipython-input-11-8247c03c3418>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  taxis_amarillos_junio_limpio['passenger_count']=taxis_amarillos_junio_limpio['passenger_count'].astype('int8')
<ipython-input-11-8247c03c3418>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  taxis_amarillos_junio_limpio['RatecodeID']=taxis_amarillos_junio_limpio['RatecodeID'].astype('int8')
<ipython-input-11-8247c03c3418>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee
0,1,2023-06-01 00:08:48,2023-06-01 00:29:41,1,3.40,1,0,140,238,1,21.9,3.50,0.5,6.70,0.0,1,33.60,2.5,0.00
1,1,2023-06-01 00:15:04,2023-06-01 00:25:18,0,3.40,1,0,50,151,1,15.6,3.50,0.5,3.00,0.0,1,23.60,2.5,0.00
2,1,2023-06-01 00:48:24,2023-06-01 01:07:07,1,10.20,1,0,138,97,1,40.8,7.75,0.5,10.00,0.0,1,60.05,0.0,1.75
3,2,2023-06-01 00:54:03,2023-06-01 01:17:29,3,9.83,1,0,100,244,1,39.4,1.00,0.5,8.88,0.0,1,53.28,2.5,0.00
4,2,2023-06-01 00:18:44,2023-06-01 00:27:18,1,1.17,1,0,137,234,1,9.3,1.00,0.5,0.72,0.0,1,15.02,2.5,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3207342,2,2023-06-30 23:37:53,2023-06-30 23:51:14,2,1.58,1,0,230,162,1,12.8,1.00,0.5,3.56,0.0,1,21.36,2.5,0.00
3207343,2,2023-06-30 23:54:47,2023-07-01 00:07:19,2,1.47,1,0,162,48,2,12.8,1.00,0.5,0.00,0.0,1,17.80,2.5,0.00
3207344,1,2023-06-30 23:47:32,2023-07-01 00:06:45,1,3.30,1,0,249,143,1,17.7,3.50,0.5,1.00,0.0,1,23.70,2.5,0.00
3207345,2,2023-06-30 23:04:54,2023-06-30 23:30:59,1,10.59,1,0,132,39,1,42.9,1.00,0.5,5.00,0.0,1,52.15,0.0,1.75


In [12]:
taxis_amarillos_abril_limpio

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee
0,1,2023-04-01 00:14:49,2023-04-01 00:45:01,2,4.90,1,0,48,223,1,28.9,3.50,0.5,6.00,0.00,1,39.90,2.5,0.00
1,2,2023-04-01 00:00:24,2023-04-01 00:56:19,1,21.89,2,0,132,43,2,70.0,0.00,0.5,0.00,6.55,1,81.80,2.5,1.25
2,1,2023-04-01 00:03:50,2023-04-01 00:14:42,2,1.30,1,0,148,113,1,11.4,3.50,0.5,2.00,0.00,1,18.40,2.5,0.00
3,1,2023-04-01 00:53:18,2023-04-01 01:01:28,1,1.50,1,0,249,79,1,10.0,3.50,0.5,1.00,0.00,1,16.00,2.5,0.00
4,2,2023-04-01 00:07:00,2023-04-01 00:17:16,2,1.49,1,0,158,246,1,11.4,1.00,0.5,1.00,0.00,1,17.40,2.5,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3197555,1,2023-04-30 23:29:24,2023-04-30 23:54:57,3,9.70,1,0,138,151,1,40.8,7.75,0.5,10.00,6.55,1,66.60,0.0,1.75
3197556,2,2023-04-30 23:03:33,2023-04-30 23:09:26,1,1.32,1,0,158,246,1,8.6,1.00,0.5,1.36,0.00,1,14.96,2.5,0.00
3197557,2,2023-04-30 23:24:22,2023-04-30 23:29:17,1,1.23,1,0,48,246,1,7.9,1.00,0.5,2.58,0.00,1,15.48,2.5,0.00
3197558,2,2023-04-30 23:39:17,2023-04-30 23:50:25,1,2.68,1,0,48,79,1,14.2,1.00,0.5,3.84,0.00,1,23.04,2.5,0.00


In [13]:
taxis_amarillos_mayo_limpio

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee
0,1,2023-05-01 00:33:13,2023-05-01 00:53:01,0,7.80,1,0,138,43,1,33.8,7.75,0.5,8.60,0.00,1,51.65,0.0,1.75
1,1,2023-05-01 00:42:49,2023-05-01 01:11:18,2,8.10,1,0,138,262,1,35.9,10.25,0.5,9.50,0.00,1,57.15,2.5,1.75
2,1,2023-05-01 00:56:34,2023-05-01 01:13:39,2,9.10,1,0,138,141,1,35.2,10.25,0.5,10.70,6.55,1,64.20,2.5,1.75
3,2,2023-05-01 00:00:52,2023-05-01 00:20:12,1,8.21,1,0,138,140,1,33.1,6.00,0.5,2.24,0.00,1,47.09,2.5,1.75
4,1,2023-05-01 00:05:50,2023-05-01 00:19:41,0,7.90,1,0,138,263,1,31.0,10.25,0.5,9.85,6.55,1,59.15,2.5,1.75
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3411848,2,2023-05-31 23:39:27,2023-05-31 23:56:59,2,3.29,1,0,239,74,1,19.8,1.00,0.5,4.96,0.00,1,29.76,2.5,0.00
3411849,2,2023-05-31 23:15:28,2023-05-31 23:24:48,1,1.81,1,0,79,234,1,12.1,1.00,0.5,2.56,0.00,1,19.66,2.5,0.00
3411850,2,2023-05-31 23:27:31,2023-06-01 00:03:00,1,6.91,1,0,186,223,1,36.6,1.00,0.5,10.40,0.00,1,52.00,2.5,0.00
3411851,1,2023-05-31 23:42:24,2023-05-31 23:53:28,1,2.50,1,0,170,114,1,13.5,3.50,0.5,3.70,0.00,1,22.20,2.5,0.00


In [14]:
taxis_amarillos_junio_limpio

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee
0,1,2023-06-01 00:08:48,2023-06-01 00:29:41,1,3.40,1,0,140,238,1,21.9,3.50,0.5,6.70,0.0,1,33.60,2.5,0.00
1,1,2023-06-01 00:15:04,2023-06-01 00:25:18,0,3.40,1,0,50,151,1,15.6,3.50,0.5,3.00,0.0,1,23.60,2.5,0.00
2,1,2023-06-01 00:48:24,2023-06-01 01:07:07,1,10.20,1,0,138,97,1,40.8,7.75,0.5,10.00,0.0,1,60.05,0.0,1.75
3,2,2023-06-01 00:54:03,2023-06-01 01:17:29,3,9.83,1,0,100,244,1,39.4,1.00,0.5,8.88,0.0,1,53.28,2.5,0.00
4,2,2023-06-01 00:18:44,2023-06-01 00:27:18,1,1.17,1,0,137,234,1,9.3,1.00,0.5,0.72,0.0,1,15.02,2.5,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3207342,2,2023-06-30 23:37:53,2023-06-30 23:51:14,2,1.58,1,0,230,162,1,12.8,1.00,0.5,3.56,0.0,1,21.36,2.5,0.00
3207343,2,2023-06-30 23:54:47,2023-07-01 00:07:19,2,1.47,1,0,162,48,2,12.8,1.00,0.5,0.00,0.0,1,17.80,2.5,0.00
3207344,1,2023-06-30 23:47:32,2023-07-01 00:06:45,1,3.30,1,0,249,143,1,17.7,3.50,0.5,1.00,0.0,1,23.70,2.5,0.00
3207345,2,2023-06-30 23:04:54,2023-06-30 23:30:59,1,10.59,1,0,132,39,1,42.9,1.00,0.5,5.00,0.0,1,52.15,0.0,1.75


##veo cuantos quedaron en la tabla##

In [15]:
print(taxis_amarillos_abril_limpio.count())

VendorID                 3197560
tpep_pickup_datetime     3197560
tpep_dropoff_datetime    3197560
passenger_count          3197560
trip_distance            3197560
RatecodeID               3197560
store_and_fwd_flag       3197560
PULocationID             3197560
DOLocationID             3197560
payment_type             3197560
fare_amount              3197560
extra                    3197560
mta_tax                  3197560
tip_amount               3197560
tolls_amount             3197560
improvement_surcharge    3197560
total_amount             3197560
congestion_surcharge     3197560
Airport_fee              3197560
dtype: int64


In [16]:
print(taxis_amarillos_mayo_limpio.count())

VendorID                 3411853
tpep_pickup_datetime     3411853
tpep_dropoff_datetime    3411853
passenger_count          3411853
trip_distance            3411853
RatecodeID               3411853
store_and_fwd_flag       3411853
PULocationID             3411853
DOLocationID             3411853
payment_type             3411853
fare_amount              3411853
extra                    3411853
mta_tax                  3411853
tip_amount               3411853
tolls_amount             3411853
improvement_surcharge    3411853
total_amount             3411853
congestion_surcharge     3411853
Airport_fee              3411853
dtype: int64


In [17]:
print(taxis_amarillos_junio_limpio.count())

VendorID                 3207347
tpep_pickup_datetime     3207347
tpep_dropoff_datetime    3207347
passenger_count          3207347
trip_distance            3207347
RatecodeID               3207347
store_and_fwd_flag       3207347
PULocationID             3207347
DOLocationID             3207347
payment_type             3207347
fare_amount              3207347
extra                    3207347
mta_tax                  3207347
tip_amount               3207347
tolls_amount             3207347
improvement_surcharge    3207347
total_amount             3207347
congestion_surcharge     3207347
Airport_fee              3207347
dtype: int64


#**Analisis Exploratorio**#

 **a) Cuantitativas**


  
  VendorID
  passenger_count
  trip_distance
  RatecodeID
  PULocationID
  DOLocationID
  payment_type
  fare_amount
  extra	mta_tax
  tip_amount
  tolls_amount
  improvement_surcharge
  total_amount
  congestion_surcharge
  Airport_fee

  sobre el dataset LIMPIO

In [18]:
print("estadisticas de la tabla de abril")
taxis_amarillos_abril_limpio_para_est=taxis_amarillos_abril_limpio.loc[:, ["VendorID", "passenger_count","trip_distance","RatecodeID","PULocationID","DOLocationID","payment_type","fare_amount","extra","mta_tax","tip_amount","tolls_amount",	"improvement_surcharge","total_amount","congestion_surcharge","Airport_fee"]]
taxis_amarillos_abril_limpio_para_est_cuali=taxis_amarillos_abril_limpio.loc[:, ["tpep_pickup_datetime", "tpep_dropoff_datetime","store_and_fwd_flag"]]
taxis_amarillos_abril_limpio_para_est.describe()

estadisticas de la tabla de abril


,VendorID,passenger_count,trip_distance,RatecodeID,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee
count,3.197560e+06,3.197560e+06,3.197560e+06,3.197560e+06,3.197560e+06,3.197560e+06,3.197560e+06,3.197560e+06,3.197560e+06,3.197560e+06,3.197560e+06,3.197560e+06,3.197560e+06,3.197560e+06,3.197560e+06,3.197560e+06
mean,1.729881e+00,1.382820e+00,3.559276e+00,1.598646e+00,1.653541e+02,1.642602e+02,1.227809e+00,1.928252e+01,1.608886e+00,4.864885e-01,3.512825e+00,5.841512e-01,9.804110e-01,2.821494e+01,2.270376e+00,1.409284e-01
std,4.440214e-01,9.064290e-01,2.321620e+01,7.148677e+00,6.390828e+01,7.006590e+01,5.118352e-01,1.893947e+01,1.848135e+00,1.054502e-01,4.073062e+00,2.172028e+00,1.944482e-01,2.363194e+01,7.835004e-01,4.756554e-01
min,1.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,-8.000000e+02,-7.500000e+00,-5.000000e-01,-9.496000e+01,-6.310000e+01,-1.000000e+00,-8.075500e+02,-2.500000e+00,-1.750000e+00
25%,1.000000e+00,1.000000e+00,1.080000e+00,1.000000e+00,1.320000e+02,1.130000e+02,1.000000e+00,9.300000e+00,0.000000e+00,5.000000e-01,1.000000e+00,0.000000e+00,1.000000e+00,1.570000e+01,2.500000e+00,0.000000e+00
50%,2.000000e+00,1.000000e+00,1.800000e+00,1.000000e+00,1.620000e+02,1.620000e+02,1.000000e+00,1.350000e+01,1.000000e+00,5.000000e-01,2.800000e+00,0.000000e+00,1.000000e+00,2.064000e+01,2.500000e+00,0.000000e+00
75%,2.000000e+00,1.000000e+00,3.400000e+00,1.000000e+00,2.340000e+02,2.340000e+02,1.000000e+00,2.120000e+01,2.500000e+00,5.000000e-01,4.380000e+00,0.000000e+00,1.000000e+00,3.024000e+01,2.500000e+00,0.000000e+00
max,2.000000e+00,9.000000e+00,9.683850e+03,9.900000e+01,2.650000e+02,2.650000e+02,4.000000e+00,2.449500e+03,9.638000e+01,4.000000e+00,4.400800e+02,1.430000e+02,1.000000e+00,2.451000e+03,2.500000e+00,1.750000e+00


In [19]:
print("estadisticas de la tabla de mayo")
taxis_amarillos_mayo_limpio_para_est=taxis_amarillos_mayo_limpio.loc[:, ["VendorID", "passenger_count","trip_distance","RatecodeID","PULocationID","DOLocationID","payment_type","fare_amount","extra","mta_tax","tip_amount","tolls_amount",	"improvement_surcharge","total_amount","congestion_surcharge","Airport_fee"]]
taxis_amarillos_mayo_limpio_para_est_cuali=taxis_amarillos_mayo_limpio.loc[:, ["tpep_pickup_datetime", "tpep_dropoff_datetime","store_and_fwd_flag"]]
taxis_amarillos_mayo_limpio_para_est.describe()

estadisticas de la tabla de mayo


,VendorID,passenger_count,trip_distance,RatecodeID,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee
count,3.411853e+06,3.411853e+06,3.411853e+06,3.411853e+06,3.411853e+06,3.411853e+06,3.411853e+06,3.411853e+06,3.411853e+06,3.411853e+06,3.411853e+06,3.411853e+06,3.411853e+06,3.411853e+06,3.411853e+06,3.411853e+06
mean,1.733914e+00,1.358800e+00,3.607005e+00,1.587358e+00,1.657744e+02,1.647855e+02,1.222692e+00,1.976695e+01,1.657985e+00,4.862339e-01,3.608628e+00,6.020575e-01,9.806445e-01,2.887243e+01,2.268303e+00,1.412078e-01
std,4.419099e-01,8.796108e-01,3.056245e+01,7.071919e+00,6.368401e+01,6.982491e+01,5.100051e-01,1.929704e+01,1.878746e+00,1.060315e-01,4.155540e+00,2.202720e+00,1.939103e-01,2.407141e+01,7.862605e-01,4.830335e-01
min,1.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,-8.700000e+02,-7.500000e+00,-5.000000e-01,-4.110000e+02,-4.655000e+01,-1.000000e+00,-9.005000e+02,-2.500000e+00,-1.750000e+00
25%,1.000000e+00,1.000000e+00,1.080000e+00,1.000000e+00,1.320000e+02,1.140000e+02,1.000000e+00,9.300000e+00,0.000000e+00,5.000000e-01,1.000000e+00,0.000000e+00,1.000000e+00,1.596000e+01,2.500000e+00,0.000000e+00
50%,2.000000e+00,1.000000e+00,1.800000e+00,1.000000e+00,1.620000e+02,1.620000e+02,1.000000e+00,1.350000e+01,1.000000e+00,5.000000e-01,2.880000e+00,0.000000e+00,1.000000e+00,2.130000e+01,2.500000e+00,0.000000e+00
75%,2.000000e+00,1.000000e+00,3.440000e+00,1.000000e+00,2.340000e+02,2.340000e+02,1.000000e+00,2.260000e+01,2.500000e+00,5.000000e-01,4.500000e+00,0.000000e+00,1.000000e+00,3.110000e+01,2.500000e+00,0.000000e+00
max,2.000000e+00,9.000000e+00,1.305347e+04,9.900000e+01,2.650000e+02,2.650000e+02,4.000000e+00,6.300900e+03,1.625000e+01,4.000000e+00,7.000000e+02,1.320400e+02,1.000000e+00,6.304900e+03,2.500000e+00,1.750000e+00


In [20]:
print("estadisticas de la tabla de junio")
taxis_amarillos_junio_limpio_para_est=taxis_amarillos_junio_limpio.loc[:, ["VendorID", "passenger_count","trip_distance","RatecodeID","PULocationID","DOLocationID","payment_type","fare_amount","extra","mta_tax","tip_amount","tolls_amount",	"improvement_surcharge","total_amount","congestion_surcharge","Airport_fee"]]
taxis_amarillos_junio_limpio_para_est_cuali=taxis_amarillos_junio_limpio.loc[:, ["tpep_pickup_datetime", "tpep_dropoff_datetime","store_and_fwd_flag"]]
taxis_amarillos_junio_limpio_para_est.describe()

estadisticas de la tabla de junio


,VendorID,passenger_count,trip_distance,RatecodeID,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee
count,3.207347e+06,3.207347e+06,3.207347e+06,3.207347e+06,3.207347e+06,3.207347e+06,3.207347e+06,3.207347e+06,3.207347e+06,3.207347e+06,3.207347e+06,3.207347e+06,3.207347e+06,3.207347e+06,3.207347e+06,3.207347e+06
mean,1.735382e+00,1.369012e+00,3.670147e+00,1.547159e+00,1.650863e+02,1.641188e+02,1.224097e+00,1.986512e+01,1.664416e+00,4.857143e-01,3.597233e+00,5.985374e-01,9.801998e-01,2.896939e+01,2.262730e+00,1.534284e-01
std,4.411299e-01,8.959539e-01,4.132603e+01,6.762597e+00,6.370443e+01,6.996401e+01,5.140086e-01,2.169519e+02,1.875703e+00,1.079281e-01,4.220875e+00,2.236137e+00,1.968406e-01,2.174386e+02,7.960739e-01,5.027511e-01
min,1.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,-7.500000e+02,-7.500000e+00,-5.000000e-01,-9.500000e+01,-7.255000e+01,-1.000000e+00,-7.510000e+02,-2.500000e+00,-1.750000e+00
25%,1.000000e+00,1.000000e+00,1.070000e+00,1.000000e+00,1.320000e+02,1.140000e+02,1.000000e+00,9.300000e+00,0.000000e+00,5.000000e-01,1.000000e+00,0.000000e+00,1.000000e+00,1.596000e+01,2.500000e+00,0.000000e+00
50%,2.000000e+00,1.000000e+00,1.800000e+00,1.000000e+00,1.620000e+02,1.620000e+02,1.000000e+00,1.350000e+01,1.000000e+00,5.000000e-01,2.860000e+00,0.000000e+00,1.000000e+00,2.100000e+01,2.500000e+00,0.000000e+00
75%,2.000000e+00,1.000000e+00,3.460000e+00,1.000000e+00,2.330000e+02,2.340000e+02,1.000000e+00,2.190000e+01,2.500000e+00,5.000000e-01,4.480000e+00,0.000000e+00,1.000000e+00,3.100000e+01,2.500000e+00,0.000000e+00
max,2.000000e+00,9.000000e+00,4.084768e+04,9.900000e+01,2.650000e+02,2.650000e+02,5.000000e+00,3.869836e+05,1.425000e+01,5.750000e+00,7.118800e+02,6.655600e+02,1.000000e+00,3.869876e+05,2.500000e+00,1.750000e+00


#**b) cualitativas**#

las 3 que no cuentan cantidades son:

# **Mes de Abril **
* tpep_pickup_datetime: desde 2023-04-01 00:14:49 al 2023-04-30 23:04:08  
* tpep_dropoff_datetime: desde 2023-04-01 00:45:01 al 2023-04-30 23:06:53
* store_and_fwd_flag: contiene 'Y'=1 o 'N'=0

# **Mes de Mayo **
* tpep_pickup_datetime: desde 2023-05-01 00:33:13	al 2023-05-31 23:50:28  
* tpep_dropoff_datetime: desde 2023-05-01 00:53:01	al 2023-06-01 00:05:44
* store_and_fwd_flag: contiene 'Y'=1 o 'N'=0

# **Mes de Junio **
* tpep_pickup_datetime: desde 2023-06-01 00:08:48	al 2023-06-30 23:06:29
* tpep_dropoff_datetime: desde 2023-06-01 00:29:41	al 2023-06-30 23:16:07
* store_and_fwd_flag: contiene 'Y'=1 o 'N'=0

a)
Exploración Inicial : analizar cada variable, considerando los siguientes aspectos
* Variables Cuantitativas: calcular medidas de resumen, media, mediana, moda, etc. **HECHO**
* Variables Cualitativas: mostrar cantidad de valores posibles, y frecuencias de cada uno **se agrupa por dia del mes, luego se contara la cantidad de apariciones de cada rango**.
* Realizar un análisis gráfico de las distribuciones de las variables más relevantes
* Analizar las correlaciones existentes entre las variables.

b) Datos Faltantes : analizar la presencia de datos faltantes en el dataset
* Realizar análisis de datos faltantes. Graficar para cada variable el porcentaje de datos faltantes con respecto al total del dataset. **HECHO**

Calcular el porcentaje de datos faltantes de cada registro.
* Revisar los datos faltantes o mal ingresados y tomar una decisión sobre
estos: reemplazo de valores, eliminación de registros incompletos, etc.
***Se decicio eliminar esos registros***

  **tabla de abril**
    iniciales: 3288250
    luego del procesamiento: 3197560
    nulos:90690
    % de nulos: 2,758%

  **tabla de mayo**
    iniciales: 3513649
    luego del procesamiento: 3411853
    nulos:101796
    % de nulos: 2,897%

  **tabla de junio**
    iniciales: 3307234
    luego del procesamiento: 3207347
    nulos:99887
    % de nulos: 3,020%

c) Valores atípicos : analizar la existencia de valores atípicos.
* Detectar valores atípicos en los datos tanto en forma univariada como multivariada.
Realizar gráficos que permitan visualizar los valores atípicos.
* Explicar qué características poseen los datos atípicos detectados y decidir el tratamiento a aplicar sobre los mismos.
d) Nuevos Features: analizar qué nuevas variables se pueden crear ya sea que resulten derivadas de los atributos existentes o de incorporar nuevas fuentes de datos.
e) Preguntas de investigación: se deben plantear preguntas cuyas respuestas puedan ser obtenidas a partir del análisis de los datos. Ejemplo:
* ¿Existe una manera de caracterizar los lugares más recurrentes para inicio/fin de viaje?
* ¿Cómo son los viajes típicamente en distancia y tiempo?
* ¿Cómo son los viajes típicamente en distancia y tiempo según el horario y/o el día de la semana?
f) Visualización de los datos: en esta sección se espera que puedan apoyarse en
visualizaciones, combinando distintas variables del dataset que los ayuden a responder preguntas sobre los datos.
Nota : Los ítems a, b, c, d, e y f son los mínimos requeridos para esta etapa, cada grupo puede
sumar técnicas y/o análisis adicionales.

##**tabla de abril**
aca separo el campo tpep_pickup_datetime y tpep_dropoff_datetime en 2 campos distintos



In [21]:
# Convert the 'tpep_pickup_datetime' column to datetime objects
taxis_amarillos_abril_limpio['tpep_pickup_datetime'] = pd.to_datetime(taxis_amarillos_abril_limpio['tpep_pickup_datetime'])

# Extract the date and time into separate columns
taxis_amarillos_abril_limpio['pickup_date'] = taxis_amarillos_abril_limpio['tpep_pickup_datetime'].dt.date
taxis_amarillos_abril_limpio['pickup_time'] = taxis_amarillos_abril_limpio['tpep_pickup_datetime'].dt.time

# Convert the 'tpep_dropoff_datetime' column to datetime objects
taxis_amarillos_abril_limpio['tpep_dropoff_datetime'] = pd.to_datetime(taxis_amarillos_abril_limpio['tpep_dropoff_datetime'])

# Extract the date and time into separate columns
taxis_amarillos_abril_limpio['dropoff_date'] = taxis_amarillos_abril_limpio['tpep_dropoff_datetime'].dt.date
taxis_amarillos_abril_limpio['dropoff_time'] = taxis_amarillos_abril_limpio['tpep_dropoff_datetime'].dt.time

#delete column tpep_dropoff_datetime
taxis_amarillos_abril_limpio.drop('tpep_pickup_datetime',axis='columns', inplace=True)
taxis_amarillos_abril_limpio.drop('tpep_dropoff_datetime',axis='columns', inplace=True)

# Print the updated DataFrame to check the result
taxis_amarillos_abril_limpio


<ipython-input-21-824d86ba9039>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  taxis_amarillos_abril_limpio['tpep_pickup_datetime'] = pd.to_datetime(taxis_amarillos_abril_limpio['tpep_pickup_datetime'])
<ipython-input-21-824d86ba9039>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  taxis_amarillos_abril_limpio['pickup_date'] = taxis_amarillos_abril_limpio['tpep_pickup_datetime'].dt.date
<ipython-input-21-824d86ba9039>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Dat

,VendorID,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,...,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee,pickup_date,pickup_time,dropoff_date,dropoff_time
0,1,2,4.90,1,0,48,223,1,28.9,3.50,...,6.00,0.00,1,39.90,2.5,0.00,2023-04-01,00:14:49,2023-04-01,00:45:01
1,2,1,21.89,2,0,132,43,2,70.0,0.00,...,0.00,6.55,1,81.80,2.5,1.25,2023-04-01,00:00:24,2023-04-01,00:56:19
2,1,2,1.30,1,0,148,113,1,11.4,3.50,...,2.00,0.00,1,18.40,2.5,0.00,2023-04-01,00:03:50,2023-04-01,00:14:42
3,1,1,1.50,1,0,249,79,1,10.0,3.50,...,1.00,0.00,1,16.00,2.5,0.00,2023-04-01,00:53:18,2023-04-01,01:01:28
4,2,2,1.49,1,0,158,246,1,11.4,1.00,...,1.00,0.00,1,17.40,2.5,0.00,2023-04-01,00:07:00,2023-04-01,00:17:16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3197555,1,3,9.70,1,0,138,151,1,40.8,7.75,...,10.00,6.55,1,66.60,0.0,1.75,2023-04-30,23:29:24,2023-04-30,23:54:57
3197556,2,1,1.32,1,0,158,246,1,8.6,1.00,...,1.36,0.00,1,14.96,2.5,0.00,2023-04-30,23:03:33,2023-04-30,23:09:26
3197557,2,1,1.23,1,0,48,246,1,7.9,1.00,...,2.58,0.00,1,15.48,2.5,0.00,2023-04-30,23:24:22,2023-04-30,23:29:17
3197558,2,1,2.68,1,0,48,79,1,14.2,1.00,...,3.84,0.00,1,23.04,2.5,0.00,2023-04-30,23:39:17,2023-04-30,23:50:25


##**tabla de Mayo**
aca separo el campo tpep_pickup_datetime en 2 campos distintos



In [22]:
# Convert the 'tpep_pickup_datetime' column to datetime objects
taxis_amarillos_mayo_limpio['tpep_pickup_datetime'] = pd.to_datetime(taxis_amarillos_mayo_limpio['tpep_pickup_datetime'])

# Extract the date and time into separate columns
taxis_amarillos_mayo_limpio['pickup_date'] = taxis_amarillos_mayo_limpio['tpep_pickup_datetime'].dt.date
taxis_amarillos_mayo_limpio['pickup_time'] = taxis_amarillos_mayo_limpio['tpep_pickup_datetime'].dt.time

# Convert the 'tpep_dropoff_datetime' column to datetime objects
taxis_amarillos_mayo_limpio['tpep_dropoff_datetime'] = pd.to_datetime(taxis_amarillos_mayo_limpio['tpep_dropoff_datetime'])

# Extract the date and time into separate columns
taxis_amarillos_mayo_limpio['dropoff_date'] = taxis_amarillos_mayo_limpio['tpep_dropoff_datetime'].dt.date
taxis_amarillos_mayo_limpio['dropoff_time'] = taxis_amarillos_mayo_limpio['tpep_dropoff_datetime'].dt.time

#delete column tpep_dropoff_datetime
taxis_amarillos_mayo_limpio.drop('tpep_pickup_datetime',axis='columns', inplace=True)
taxis_amarillos_mayo_limpio.drop('tpep_dropoff_datetime',axis='columns', inplace=True)

# Print the updated DataFrame to check the result
taxis_amarillos_mayo_limpio

<ipython-input-22-893f2331714a>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  taxis_amarillos_mayo_limpio['tpep_pickup_datetime'] = pd.to_datetime(taxis_amarillos_mayo_limpio['tpep_pickup_datetime'])
<ipython-input-22-893f2331714a>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  taxis_amarillos_mayo_limpio['pickup_date'] = taxis_amarillos_mayo_limpio['tpep_pickup_datetime'].dt.date
<ipython-input-22-893f2331714a>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFra

,VendorID,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,...,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee,pickup_date,pickup_time,dropoff_date,dropoff_time
0,1,0,7.80,1,0,138,43,1,33.8,7.75,...,8.60,0.00,1,51.65,0.0,1.75,2023-05-01,00:33:13,2023-05-01,00:53:01
1,1,2,8.10,1,0,138,262,1,35.9,10.25,...,9.50,0.00,1,57.15,2.5,1.75,2023-05-01,00:42:49,2023-05-01,01:11:18
2,1,2,9.10,1,0,138,141,1,35.2,10.25,...,10.70,6.55,1,64.20,2.5,1.75,2023-05-01,00:56:34,2023-05-01,01:13:39
3,2,1,8.21,1,0,138,140,1,33.1,6.00,...,2.24,0.00,1,47.09,2.5,1.75,2023-05-01,00:00:52,2023-05-01,00:20:12
4,1,0,7.90,1,0,138,263,1,31.0,10.25,...,9.85,6.55,1,59.15,2.5,1.75,2023-05-01,00:05:50,2023-05-01,00:19:41
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3411848,2,2,3.29,1,0,239,74,1,19.8,1.00,...,4.96,0.00,1,29.76,2.5,0.00,2023-05-31,23:39:27,2023-05-31,23:56:59
3411849,2,1,1.81,1,0,79,234,1,12.1,1.00,...,2.56,0.00,1,19.66,2.5,0.00,2023-05-31,23:15:28,2023-05-31,23:24:48
3411850,2,1,6.91,1,0,186,223,1,36.6,1.00,...,10.40,0.00,1,52.00,2.5,0.00,2023-05-31,23:27:31,2023-06-01,00:03:00
3411851,1,1,2.50,1,0,170,114,1,13.5,3.50,...,3.70,0.00,1,22.20,2.5,0.00,2023-05-31,23:42:24,2023-05-31,23:53:28


##**tabla de Junio**
aca separo el campo tpep_pickup_datetime en 2 campos distintos



In [23]:
# Convert the 'tpep_pickup_datetime' column to datetime objects
taxis_amarillos_junio_limpio['tpep_pickup_datetime'] = pd.to_datetime(taxis_amarillos_junio_limpio['tpep_pickup_datetime'])

# Extract the date and time into separate columns
taxis_amarillos_junio_limpio['pickup_date'] = taxis_amarillos_junio_limpio['tpep_pickup_datetime'].dt.date
taxis_amarillos_junio_limpio['pickup_time'] = taxis_amarillos_junio_limpio['tpep_pickup_datetime'].dt.time

# Convert the 'tpep_dropoff_datetime' column to datetime objects
taxis_amarillos_junio_limpio['tpep_dropoff_datetime'] = pd.to_datetime(taxis_amarillos_junio_limpio['tpep_dropoff_datetime'])

# Extract the date and time into separate columns
taxis_amarillos_junio_limpio['dropoff_date'] = taxis_amarillos_junio_limpio['tpep_dropoff_datetime'].dt.date
taxis_amarillos_junio_limpio['dropoff_time'] = taxis_amarillos_junio_limpio['tpep_dropoff_datetime'].dt.time

#delete column tpep_dropoff_datetime
taxis_amarillos_junio_limpio.drop('tpep_pickup_datetime',axis='columns', inplace=True)
taxis_amarillos_junio_limpio.drop('tpep_dropoff_datetime',axis='columns', inplace=True)

# Print the updated DataFrame to check the result
taxis_amarillos_junio_limpio

<ipython-input-23-b09780e15818>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  taxis_amarillos_junio_limpio['tpep_pickup_datetime'] = pd.to_datetime(taxis_amarillos_junio_limpio['tpep_pickup_datetime'])
<ipython-input-23-b09780e15818>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  taxis_amarillos_junio_limpio['pickup_date'] = taxis_amarillos_junio_limpio['tpep_pickup_datetime'].dt.date
<ipython-input-23-b09780e15818>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Dat

,VendorID,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,...,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee,pickup_date,pickup_time,dropoff_date,dropoff_time
0,1,1,3.40,1,0,140,238,1,21.9,3.50,...,6.70,0.0,1,33.60,2.5,0.00,2023-06-01,00:08:48,2023-06-01,00:29:41
1,1,0,3.40,1,0,50,151,1,15.6,3.50,...,3.00,0.0,1,23.60,2.5,0.00,2023-06-01,00:15:04,2023-06-01,00:25:18
2,1,1,10.20,1,0,138,97,1,40.8,7.75,...,10.00,0.0,1,60.05,0.0,1.75,2023-06-01,00:48:24,2023-06-01,01:07:07
3,2,3,9.83,1,0,100,244,1,39.4,1.00,...,8.88,0.0,1,53.28,2.5,0.00,2023-06-01,00:54:03,2023-06-01,01:17:29
4,2,1,1.17,1,0,137,234,1,9.3,1.00,...,0.72,0.0,1,15.02,2.5,0.00,2023-06-01,00:18:44,2023-06-01,00:27:18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3207342,2,2,1.58,1,0,230,162,1,12.8,1.00,...,3.56,0.0,1,21.36,2.5,0.00,2023-06-30,23:37:53,2023-06-30,23:51:14
3207343,2,2,1.47,1,0,162,48,2,12.8,1.00,...,0.00,0.0,1,17.80,2.5,0.00,2023-06-30,23:54:47,2023-07-01,00:07:19
3207344,1,1,3.30,1,0,249,143,1,17.7,3.50,...,1.00,0.0,1,23.70,2.5,0.00,2023-06-30,23:47:32,2023-07-01,00:06:45
3207345,2,1,10.59,1,0,132,39,1,42.9,1.00,...,5.00,0.0,1,52.15,0.0,1.75,2023-06-30,23:04:54,2023-06-30,23:30:59


#discretizo horas
rango 1: 0:00:00 a 0:59:59

rango 2: 1:00:00 a 1:59:59

rango 3: 2:00:00 a 2:59:59

rango 4: 3:00:00 a 3:59:59

rango 5: 4:00:00 a 4:59:59

rango 6: 5:00:00 a 5:59:59

rango 7: 6:00:00 a 6:59:59

rango 8: 7:00:00 a 7:59:59

rango 9: 8:00:00 a 8:59:59

rango 10: 9:00:00 a 9:59:59

rango 11: 10:00:00 a 10:59:59

rango 12: 11:00:00 a 11:59:59

rango 13: 12:00:00 a 12:59:59

rango 14: 13:00:00 a 13:59:59

rango 15: 14:00:00 a 14:59:59

rango 16: 15:00:00 a 15:59:59

rango 17: 16:00:00 a 16:59:59

rango 18: 17:00:00 a 17:59:59

rango 19: 18:00:00 a 18:59:59

rango 20: 19:00:00 a 19:59:59

rango 21: 20:00:00 a 20:59:59

rango 22: 21:00:00 a 21:59:59

rango 23: 22:00:00 a 22:59:59

rango 24: 23:00:00 a 23:59:59



#funcion de discretizacion de horas

In [73]:
import pandas as pd
def de_hora_a_rango_pickup(tabla):
#  for indice in range(len(tabla)):
#    tabla.iloc[indice,'pickup_time_range']=tabla.iloc[indice,'pickup_time'].hour()
  for indice in range(len(tabla)):
    # Access the 'pickup_time' value using .iloc
    pickup_time = tabla.iloc[indice, tabla.columns.get_loc('pickup_time')]
    # Extract the hour from the Time object
    hour_value = pickup_time.hour
    # Use .loc to access and set the value in the new column
    tabla.loc[indice, 'pickup_time_range'] = hour_value

In [74]:
de_hora_a_rango_pickup(taxis_amarillos_abril_limpio)
taxis_amarillos_abril_limpio

,VendorID,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,...,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee,pickup_date,pickup_time,dropoff_date,dropoff_time,pickup_time_range
0,1,2,4.90,1,0,48,223,1,28.9,3.50,...,0.00,1,39.90,2.5,0.00,2023-04-01,00:14:49,2023-04-01,00:45:01,0
1,2,1,21.89,2,0,132,43,2,70.0,0.00,...,6.55,1,81.80,2.5,1.25,2023-04-01,00:00:24,2023-04-01,00:56:19,0
2,1,2,1.30,1,0,148,113,1,11.4,3.50,...,0.00,1,18.40,2.5,0.00,2023-04-01,00:03:50,2023-04-01,00:14:42,0
3,1,1,1.50,1,0,249,79,1,10.0,3.50,...,0.00,1,16.00,2.5,0.00,2023-04-01,00:53:18,2023-04-01,01:01:28,0
4,2,2,1.49,1,0,158,246,1,11.4,1.00,...,0.00,1,17.40,2.5,0.00,2023-04-01,00:07:00,2023-04-01,00:17:16,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3197555,1,3,9.70,1,0,138,151,1,40.8,7.75,...,6.55,1,66.60,0.0,1.75,2023-04-30,23:29:24,2023-04-30,23:54:57,23
3197556,2,1,1.32,1,0,158,246,1,8.6,1.00,...,0.00,1,14.96,2.5,0.00,2023-04-30,23:03:33,2023-04-30,23:09:26,23
3197557,2,1,1.23,1,0,48,246,1,7.9,1.00,...,0.00,1,15.48,2.5,0.00,2023-04-30,23:24:22,2023-04-30,23:29:17,23
3197558,2,1,2.68,1,0,48,79,1,14.2,1.00,...,0.00,1,23.04,2.5,0.00,2023-04-30,23:39:17,2023-04-30,23:50:25,23


In [75]:
def de_hora_a_rango_dropoff_time(tabla):
#  for indice in range(len(tabla)):
#    tabla.iloc[indice,'pickup_time_range']=tabla.iloc[indice,'pickup_time'].hour()
  for indice in range(len(tabla)):
    # Access the 'pickup_time' value using .iloc
    pickup_time = tabla.iloc[indice, tabla.columns.get_loc('dropoff_time')]
    # Extract the hour from the Time object
    hour_value = pickup_time.hour
    # Use .loc to access and set the value in the new column
    tabla.loc[indice, 'dropoff_time_range'] = hour_value

In [ ]:
de_hora_a_rango_dropoff_time(taxis_amarillos_abril_limpio)
taxis_amarillos_abril_limpio

<ipython-input-75-dbfe0c31701d>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tabla.loc[indice, 'dropoff_time_range'] = hour_value
